In [1]:
import gym
import JSSEnv # an ongoing issue with OpenAi's gym causes it to not import automatically external modules, see: https://github.com/openai/gym/issues/2809
from wrapper.jssp_wrapper_klagenfurt import jssp_klagenfurt_obs_wrapper
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer
import gym
from ray import tune
from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel
from ray.rllib.models.catalog import ModelCatalog


In [3]:
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.registry import register_env
import JSSEnv
# Configure the algorithm.
def env_creator(env_config):
    import JSSEnv
    env = jssp_klagenfurt_obs_wrapper(gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/abz8.txt'}))
    return env

# use tune to register the custom environment for the ppo trainer
register_env('jssp_klagenfurt',env_creator)

config = {
    "env": "jssp_klagenfurt",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 1,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    #"horizon":1,
    "evaluation_duration":10,
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": True,
    },
    
}


In [ ]:

trainer = PPOTrainer(config=config)
import time

print("start training")
for _ in range(0,200):
    tmp=time.time()
    trainer.train()
    print(f"training iteration {_} finished after {time.time() - tmp} seconds")
    trainer.save_checkpoint(f"training_checkpoints/checkpoints_ppo_jsp_klagenfurt")


In [4]:
env = env_creator('test')

print(env.observation_space['obs'].shape)
print(env.action_space.n)

(21, 7)


In [ ]:
episode_reward = 0
env = env_creator('test')

done = False
obs = env.reset()
iterations=0
env.render()
while not done:
    print(f"start with {iterations} iteration and episode reward {episode_reward}")
    action = trainer.compute_action(obs)
    #print(f"action: {action}")
    obs, reward, done, info = env.step(action)
    episode_reward += reward
    env.render()
    iterations += 1
env.render()
env.close()
print(episode_reward)
print(iterations)

In [ ]:
#print(obs)
print(obs['action_mask'][action])

In [ ]:

train_agent=True
config = {
    "framework": "torch",
    "disable_env_checking":False,
    "num_workers"       : 4,
    "rollout_fragment_length": 50,
    "train_batch_size"  : 500,
    "sgd_minibatch_size": 64,
    "lr"                : 0.0001,
    "num_sgd_iter"      : 1,
    "mcts_config"       : {
        "puct_coefficient"   : 1.5,
        "num_simulations"    : 100,
        "temperature"        : 1.0,
        "dirichlet_epsilon"  : 0.20,
        "dirichlet_noise"    : 0.03,
        "argmax_tree_policy" : False,
        "add_dirichlet_noise": True,
    },
    "ranked_rewards"    : {
        "enable": True,
    },
    "model"             : {
        "custom_model": "dense_model",

    },
}

def env_creator(config):
    import JSSEnv
    env = jssp_klagenfurt_obs_wrapper(gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/abz8.txt'}))
    return env

ModelCatalog.register_custom_model("dense_model", DenseModel)    
tune.register_env('jssp_klagenfurt',env_creator)

agent = AlphaZeroTrainer( config=config, env='jssp_klagenfurt')


In [ ]:
import time
if train_agent:
    print("start training")
    for _ in range(0,1):
        tmp=time.time()
        agent.train()
        print(f"training iteration {_} finished after {time.time() - tmp} seconds")
        #agent.save(f"save_az/rllib_checkpoint{_}")
        agent.save_checkpoint(f"training_checkpoints/checkpoints_az_jsp_klagenfurt")


In [ ]:
import numpy as np
legal_action=state['action_mask']
action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
new_state, reward, done, info=env.step(action)
env.render()


In [ ]:
import copy
finished_after=[]
for j in range(1):
    state=env.reset()
    for i in range(9999):
        legal_action=state['action_mask']
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        state, reward, done, info=env.step(action)
        if done==True:
            print(f"finished! no legal actions possible after {i} steps")
            finished_after.append(i)
            break

        #sleep(1)
import time
env.render()
x=time.time()
env_tmp=copy.deepcopy(env)
y=time.time()
print(y-x)
env.render()

In [ ]:
finished_after.sort()

from matplotlib import pyplot as plt
plt.plot(finished_after)